In [1]:
# You only need to run this section once.

%load_ext autoreload
%autoreload 2

import os

print("original dir: ", os.getcwd())

new_path = "../"
os.chdir(new_path)

print("changed dir: ", os.getcwd())

original dir:  d:\我的\大学\3秋\学业\创新实践\repo\Nonlinear-Erasure-Code\src\test
changed dir:  d:\我的\大学\3秋\学业\创新实践\repo\Nonlinear-Erasure-Code\src


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from base_model.LeNet5 import LeNet5
from base_model.VGG16 import VGG16
from base_model.ResNet18 import ResNet18


# TODO: 完成该节中的设置


# 定义模型

# # LeNet5
# model_name = "LeNet5"
# model = LeNet5(input_dim=(1, 28, 28), num_classes=10)

# # VGG16
# model_name = "VGG16"
# model = VGG16(input_dim=(3, 32, 32), num_classes=10)
# model = VGG16(input_dim=(3, 32, 32), num_classes=100)

# ResNet
model_name = "ResNet18"
model = ResNet18(input_dim=(3, 32, 32), num_classes=10)
# model = ResNet18(input_dim=(3, 32, 32), num_classes=100)


# 设置数据转换
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)
# transform = transforms.Compose(
#     [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
# )
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomRotation(15),
#     transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])


# 设置数据集（训练集与测试集合）

"""
MNIST:
image: (1, 28, 28), label: (0-9)
train_dataset: [(image, label), (image, label), ...], len(train_dataset): 60000
test_dataset: [(image, label), (image, label), ...], len(test_dataset): 10000

CIFAR-10:
image: (3, 32, 32), label: (0-9)
train_dataset: [(image, label), (image, label), ...], len(train_dataset): 50000
test_dataset: [(image, label), (image, label), ...], len(test_dataset): 10000

CIFAR-100:
image: (3, 32, 32), label: (0-99)
train_dataset: [(image, label), (image, label), ...], len(train_dataset): 50000
test_dataset: [(image, label), (image, label), ...], len(test_dataset): 10000
"""

# MNIST
# dataset_name = "MNIST"
# train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# CIFAR-10
dataset_name = "CIFAR10"
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# CIFAR-100
# dataset_name = "CIFAR100"
# train_dataset = datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataset = datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)
# test_loader= DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
import datetime
import os


print(f"Train dataset: {len(train_dataset)}")
print(f"Test dataset: {len(test_dataset)}")
print("image size: ", train_dataset[0][0].size())

# 部署到设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0)


# 训练循环
model.train()  # 设置模型为训练模式
num_epochs = 10  # 迭代次数
for epoch in range(num_epochs):
    # 使用 tqdm 包装训练数据加载器
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for data, target in train_loader_tqdm:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        # target 变为 one-hot 编码
        target = torch.zeros(target.size(0), 10).to(device).scatter_(1, target.view(-1, 1), 1)

        # 正向传播
        output = model(data)
        loss = criterion(output, target)

        # 反向传播和优化
        optimizer.zero_grad()  # 清除之前的梯度
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权重

        # 更新进度条的描述
        train_loader_tqdm.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


# 测试循环
model.eval()  # 设置模型为评估模式
correct = 0
total = 0
with torch.no_grad():  # 在评估过程中不计算梯度
    for data, target in test_loader:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"Accuracy on the Test set: {100 * correct / total}%")

# 保存模型
now = datetime.datetime.now()
date = now.strftime("%Y_%m_%d")
filepath = f"base_model/{model_name}/{dataset_name}/{date}/model.pth"
dirpath = os.path.dirname(filepath)
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
torch.save(model.state_dict(), filepath)

# 读取模型
# model = LeNet5(input_dim=(1, 28, 28), num_classes=10)
# model.load_state_dict(torch.load(filepath))

# gpu: 3min58s
# cpu: 5min17s

Train dataset: 50000
Test dataset: 10000
image size:  torch.Size([3, 32, 32])
Device: cuda


Epoch 1/10: 100%|██████████| 782/782 [01:54<00:00,  6.84it/s, loss=1.97]


Epoch 1/10, Loss: 1.974961280822754


Epoch 2/10: 100%|██████████| 782/782 [01:53<00:00,  6.89it/s, loss=1.89] 


Epoch 2/10, Loss: 1.8942937850952148


...

Epoch 8/10, Loss: 0.27219173312187195


Epoch 9/10: 100%|██████████| 782/782 [01:58<00:00,  6.59it/s, loss=1.11]  


Epoch 9/10, Loss: 1.1131223440170288


Epoch 10/10: 100%|██████████| 782/782 [01:58<00:00,  6.59it/s, loss=0.29]  


Epoch 10/10, Loss: 0.2899174988269806
Accuracy on the Test set: 54.27%
